# 2 Hypothesis testing - comparing the mean of 2 groups
Run a t-test to compare whether the average monthly return of AAPL is greater than the average monthly returns of MSFT

In [4]:
import pandas_datareader as pdr
import numpy as np

In [5]:
# Collecting the real data from returns of AAPL from July 2019 until July 2022 
AAPL = pdr.get_data_yahoo("AAPL", start = "2017-07-01", end = "2022-06-30", interval = "m")
AAPL

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-07-03,36.325001,35.775002,36.220001,35.875000,57111200.0,33.914539
2017-07-05,36.197498,35.680000,35.922501,36.022499,86278400.0,34.053970
2017-07-06,35.875000,35.602501,35.755001,35.682499,96515200.0,33.732559
2017-07-07,36.187500,35.724998,35.724998,36.044998,76806800.0,34.075245
2017-07-10,36.487499,35.842499,36.027500,36.264999,84362400.0,34.283211
...,...,...,...,...,...,...
2022-06-24,141.910004,139.770004,139.899994,141.660004,89116800.0,141.463501
2022-06-27,143.490005,140.970001,142.699997,141.660004,70207900.0,141.463501
2022-06-28,143.419998,137.320007,142.130005,137.440002,67083400.0,137.249359
